# Measurement Randomized Compiling

In [ ]:
from cirq import Circuit, LineQid, measure, Simulator
from qdit.compiling import Compiler
dim = 3

measurement_circuit = Circuit()
qutrit = LineQid(0, dimension = dim)
measurement_circuit.append(measure(qutrit, key="x"))
print(measurement_circuit)

In [ ]:
cycles = 100
circuits = []
for _ in range(cycles):
    
    compiler = Compiler(measurement_circuit, d=dim, mrc=True)
    compiled_circuit, corrections = compiler.compile()
    circuits.append((compiled_circuit, corrections))

    if _==0 or _ == cycles - 1:
        print(f"Cycle {_}:")
        print(compiled_circuit)
        
print("First circuit bit-flip correction:", circuits[0][1])

Apply classical corrections to the results:

In [ ]:
# Run the circuit to get results
simulator = Simulator()

for compiled_circuit, correction in circuits:
    results = simulator.run(compiled_circuit, repetitions=100)

    # Now apply the corrections
    for key, corrections in correction.items():
        # Get the measurement data as a numpy array
        data = results.measurements[key]
        # Apply corrections
        for qudit_idx, x_power in corrections:
            data[:, qudit_idx] = (data[:, qudit_idx] - x_power) % dim

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the measurement data for key "x"
measurement_data = results.measurements['x']

# Create histogram
plt.figure(figsize=(8, 6))
plt.hist(measurement_data, bins=np.arange(dim+1)-0.5, rwidth=0.8)
plt.xticks(range(dim))
plt.xlabel('Measurement Outcome')
plt.ylabel('Counts')
plt.title(f'Distribution of {len(measurement_data)} Measurements')
plt.grid(True, alpha=0.3)
plt.show()

# Print statistics
print(f"Mean: {np.mean(measurement_data):.2f}")
print(f"Distribution of outcomes:")
for i in range(dim):
    count = np.sum(measurement_data == i)
    percentage = count/len(measurement_data)*100
    print(f"  |{i}⟩: {count} times ({percentage:.1f}%)")